# Analyzing International Debt Data

In this notebook, we will analyze the international debt data collected by The World Bank. The dataset contains information about the amount of debt (in USD) owed by developing countries across several categories. The goal is to extract insights into how different countries manage their debt and understand the distribution of debt indicators across various nations.

Throughout this notebook, we will use SQL queries to interact with the dataset stored in a MySQL database. Each query is designed to answer specific questions about the dataset, ranging from summarizing the total debt to identifying the countries with the highest debt in specific categories.


## 1. Displaying the First 10 Rows of Data

To begin our analysis, we start by inspecting the first 10 rows of the `international_debt` table. This gives us an overview of the structure and content of the dataset, including the country names, indicator codes, and the corresponding debt amounts.

By limiting the result to 10 rows, we keep the output manageable while still gaining valuable insights into the nature of the data.


In [34]:
import pymysql
conn = pymysql.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "World_Debt"
)

cur = conn.cursor()

cur.execute("SELECT * FROM international_debt LIMIT 10")

result = cur.fetchall()

print(result)


(('Afghanistan', 'AFG', 'Disbursements on external debt, long-term (DIS, current US$)', 'DT.DIS.DLXF.CD', Decimal('72894453.70')), ('Afghanistan', 'AFG', 'Interest payments on external debt, long-term (INT, current US$)', 'DT.INT.DLXF.CD', Decimal('53239440.10')), ('Afghanistan', 'AFG', 'PPG, bilateral (AMT, current US$)', 'DT.AMT.BLAT.CD', Decimal('61739336.90')), ('Afghanistan', 'AFG', 'PPG, bilateral (DIS, current US$)', 'DT.DIS.BLAT.CD', Decimal('49114729.40')), ('Afghanistan', 'AFG', 'PPG, bilateral (INT, current US$)', 'DT.INT.BLAT.CD', Decimal('39903620.10')), ('Afghanistan', 'AFG', 'PPG, multilateral (AMT, current US$)', 'DT.AMT.MLAT.CD', Decimal('39107845.00')), ('Afghanistan', 'AFG', 'PPG, multilateral (DIS, current US$)', 'DT.DIS.MLAT.CD', Decimal('23779724.30')), ('Afghanistan', 'AFG', 'PPG, multilateral (INT, current US$)', 'DT.INT.MLAT.CD', Decimal('13335820.00')), ('Afghanistan', 'AFG', 'PPG, official creditors (AMT, current US$)', 'DT.AMT.OFFT.CD', Decimal('100847181.90

## 2. Counting the Total Number of Distinct Countries

This query counts the number of distinct countries represented in the `international_debt` table. By using the `COUNT(DISTINCT country_name)` function, we ensure that each country is counted only once, regardless of how many records it has in the dataset.

This information is crucial for understanding the dataset's scope, as it tells us how many unique countries are involved in the recorded debt data. Knowing the total number of distinct countries helps in contextualizing the analysis and can guide further country-specific investigations.




In [22]:
# Execute the query
cur.execute("""
    SELECT COUNT(DISTINCT country_name) AS total_distinct_countries
    FROM international_debt
""")

# Fetch the result
total_distinct_countries = cur.fetchone()

# Print the result
print(f"Total distinct countries: {total_distinct_countries[0]}")


Total distinct countries: 124


## 3. Listing All Distinct Debt Indicators

This query retrieves all distinct debt indicators from the `international_debt` table. By using the `DISTINCT` keyword, the query ensures that each `indicator_code` is listed only once, providing a unique set of debt indicators. The results are then ordered alphabetically by `indicator_code`.

This analysis is useful for understanding the range of debt categories tracked in the dataset. By listing all distinct indicators, we gain insights into the various types of financial obligations recorded, which can be important for subsequent analyses focused on specific debt types.



In [27]:
# Execute the query
cur.execute("""
    SELECT DISTINCT indicator_code AS distinct_debt_indicators
    FROM international_debt
    ORDER BY distinct_debt_indicators
""")

# Fetch all results
distinct_debt_indicators = cur.fetchall()

# Create DataFrame and display it
df = pd.DataFrame(distinct_debt_indicators, columns=["distinct_debt_indicators"])
print(df)


   distinct_debt_indicators
0            DT.AMT.BLAT.CD
1            DT.AMT.DLXF.CD
2            DT.AMT.DPNG.CD
3            DT.AMT.MLAT.CD
4            DT.AMT.OFFT.CD
5            DT.AMT.PBND.CD
6            DT.AMT.PCBK.CD
7            DT.AMT.PROP.CD
8            DT.AMT.PRVT.CD
9            DT.DIS.BLAT.CD
10           DT.DIS.DLXF.CD
11           DT.DIS.MLAT.CD
12           DT.DIS.OFFT.CD
13           DT.DIS.PCBK.CD
14           DT.DIS.PROP.CD
15           DT.DIS.PRVT.CD
16           DT.INT.BLAT.CD
17           DT.INT.DLXF.CD
18           DT.INT.DPNG.CD
19           DT.INT.MLAT.CD
20           DT.INT.OFFT.CD
21           DT.INT.PBND.CD
22           DT.INT.PCBK.CD
23           DT.INT.PROP.CD
24           DT.INT.PRVT.CD


## 4. Calculating the Total Debt in Millions

This query calculates the total debt recorded in the dataset, expressed in millions of USD. By summing the `debt` values and dividing by 1,000,000, we convert the total debt into millions. The `ROUND` function is then used to round the result to two decimal places for clarity.

This summary statistic provides a high-level overview of the total debt burden captured in the dataset, offering a sense of the scale of international debt among the countries included.


In [28]:
# Execute the query
cur.execute("""
    SELECT 
        ROUND(SUM(debt/1000000), 2) AS total_debt
    FROM international_debt
""")

# Fetch the result
total_debt = cur.fetchone()

# Print the result with column name
print(f"Total debt (in millions): {total_debt[0]} million USD")



Total debt (in millions): 3079734.49 million USD


## 5. Identifying the Country with the Highest Total Debt

This query identifies the country with the highest total debt in the dataset. By using the `SUM` function, the query calculates the total debt for each country by summing all debt entries associated with that country. The results are grouped by `country_name` and ordered in descending order of total debt, with the `LIMIT 1` clause ensuring that only the country with the highest debt is returned.

This analysis is crucial for understanding which country carries the most significant financial burden in terms of debt, highlighting potential economic challenges or areas where international financial assistance might be most needed.


In [30]:
# Execute the query
cur.execute("""
    SELECT 
        country_name, 
        SUM(debt) AS total_debt
    FROM international_debt
    GROUP BY country_name
    ORDER BY total_debt DESC
    LIMIT 1
""")

# Fetch the result
country_with_highest_debt = cur.fetchone()

# Print the result
print(f"Country with the highest total debt: {country_with_highest_debt[0]} with {country_with_highest_debt[1]:,.2f} USD")


Country with the highest total debt: China with 285,793,494,734.20 USD


## 6. Top 10 Debt Indicators with the Highest Average Debt

This query calculates the average debt associated with each debt indicator across all countries in the dataset. By using the `AVG` function, we compute the mean debt for each `indicator_code`, which is then grouped by both `indicator_code` and `indicator_name`. The results are ordered by the average debt in descending order, and the top 10 indicators are selected.

This analysis highlights which debt indicators typically involve the largest financial amounts on average, providing insights into the most significant types of debt obligations faced by countries. Understanding these indicators can help prioritize economic policy discussions and financial management strategies.


In [32]:
# Execute the query
cur.execute("""
    SELECT 
        indicator_code AS debt_indicator,
        indicator_name,
        AVG(debt) AS average_debt
    FROM international_debt
    GROUP BY debt_indicator, indicator_name
    ORDER BY average_debt DESC
    LIMIT 10
""")

# Fetch all results
top_average_debt_indicators = cur.fetchall()

# Print the results
for indicator in top_average_debt_indicators:
    print(f"{indicator[0]} - {indicator[1]}: {indicator[2]:,.2f} USD")


DT.AMT.DLXF.CD - Principal repayments on external debt, long-term (AMT, current US$): 5,904,868,401.50 USD
DT.AMT.DPNG.CD - Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$): 5,161,194,333.81 USD
DT.DIS.DLXF.CD - Disbursements on external debt, long-term (DIS, current US$): 2,152,041,216.89 USD
DT.DIS.OFFT.CD - PPG, official creditors (DIS, current US$): 1,958,983,452.86 USD
DT.AMT.PRVT.CD - PPG, private creditors (AMT, current US$): 1,803,694,101.96 USD
DT.INT.DLXF.CD - Interest payments on external debt, long-term (INT, current US$): 1,644,024,067.65 USD
DT.DIS.BLAT.CD - PPG, bilateral (DIS, current US$): 1,223,139,290.40 USD
DT.INT.DPNG.CD - Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$): 1,220,410,844.42 USD
DT.AMT.OFFT.CD - PPG, official creditors (AMT, current US$): 1,191,187,963.08 USD
DT.AMT.PBND.CD - PPG, bonds (AMT, current US$): 1,082,623,947.65 USD


## 7. Identifying the Country with the Highest Debt in a Specific Indicator

In this query, we aim to find out which country has the highest debt for a specific debt indicator, namely `DT.AMT.DLXF.CD`. This indicator represents a particular category of debt, and by using the `MAX` function within a subquery, we identify the maximum debt value associated with this indicator. The outer query then retrieves the `country_name` and `indicator_name` corresponding to this maximum debt.

This analysis is useful for pinpointing which country is most heavily indebted under this specific debt category, providing insights into the distribution of debt across different nations for particular financial obligations.


In [14]:
# Execute the query
cur.execute("""
    SELECT 
        country_name, 
        indicator_name
    FROM international_debt
    WHERE debt IN (
        SELECT 
            MAX(debt)
        FROM international_debt
        WHERE indicator_code = 'DT.AMT.DLXF.CD'
    )
""")

# Fetch all results
result = cur.fetchall()

# Print the results
for row in result:
    print(f"Country: {row[0]}, Indicator Name: {row[1]}")


Country: China, Indicator Name: Principal repayments on external debt, long-term (AMT, current US$)


## 8. Top 20 Most Frequent Debt Indicators

This query counts the occurrences of each debt indicator in the dataset, allowing us to identify the most commonly reported types of debt. The `COUNT(*)` function is used to count the number of times each `indicator_code` appears in the `international_debt` table. The results are grouped by `indicator_code` and ordered first by the frequency of occurrence in descending order, and then by the `indicator_code` in descending order as a secondary sort criterion.

By limiting the results to the top 20 indicators, we focus on the most significant debt categories, providing insights into the types of debt that are most prevalent across countries. Understanding these patterns can help in assessing the global financial landscape and identifying key areas of economic focus.


In [15]:
# Execute the query
cur.execute("""
    SELECT indicator_code, COUNT(*) AS indicator_count
    FROM international_debt
    GROUP BY indicator_code
    ORDER BY indicator_count DESC, indicator_code DESC
    LIMIT 20
""")

# Fetch all results
top_indicators = cur.fetchall()

# Print the results
for indicator in top_indicators:
    print(f"Indicator Code: {indicator[0]}, Count: {indicator[1]}")


Indicator Code: DT.INT.OFFT.CD, Count: 124
Indicator Code: DT.INT.MLAT.CD, Count: 124
Indicator Code: DT.INT.DLXF.CD, Count: 124
Indicator Code: DT.AMT.OFFT.CD, Count: 124
Indicator Code: DT.AMT.MLAT.CD, Count: 124
Indicator Code: DT.AMT.DLXF.CD, Count: 124
Indicator Code: DT.DIS.DLXF.CD, Count: 123
Indicator Code: DT.INT.BLAT.CD, Count: 122
Indicator Code: DT.DIS.OFFT.CD, Count: 122
Indicator Code: DT.AMT.BLAT.CD, Count: 122
Indicator Code: DT.DIS.MLAT.CD, Count: 120
Indicator Code: DT.DIS.BLAT.CD, Count: 113
Indicator Code: DT.INT.PRVT.CD, Count: 98
Indicator Code: DT.AMT.PRVT.CD, Count: 98
Indicator Code: DT.INT.PCBK.CD, Count: 84
Indicator Code: DT.AMT.PCBK.CD, Count: 84
Indicator Code: DT.INT.DPNG.CD, Count: 79
Indicator Code: DT.AMT.DPNG.CD, Count: 79
Indicator Code: DT.INT.PBND.CD, Count: 69
Indicator Code: DT.AMT.PBND.CD, Count: 69


## 9. Top 10 Countries with the Highest Maximum Debt

In this query, we aim to identify the top 10 countries that have the highest maximum debt recorded across all debt indicators. By using the `MAX` function, we retrieve the single largest debt value for each country. The results are then sorted in descending order to highlight the countries with the most significant individual debt amounts.

This analysis helps us understand which countries have the largest individual debt entries and may provide insights into specific economic events or financial commitments that led to such high levels of debt.


In [16]:
# Execute the query
cur.execute("""
    SELECT country_name, MAX(debt) AS maximum_debt
    FROM international_debt
    GROUP BY country_name
    ORDER BY maximum_debt DESC
    LIMIT 10
""")

# Fetch all results
top_countries_with_max_debt = cur.fetchall()

# Print the results
for country in top_countries_with_max_debt:
    print(f"Country: {country[0]}, Maximum Debt: {country[1]:,.2f} USD")


Country: China, Maximum Debt: 96,218,620,835.70 USD
Country: Brazil, Maximum Debt: 90,041,840,304.10 USD
Country: Russian Federation, Maximum Debt: 66,589,761,833.50 USD
Country: Turkey, Maximum Debt: 51,555,031,005.80 USD
Country: South Asia, Maximum Debt: 48,756,295,898.20 USD
Country: Least developed countries: UN classification, Maximum Debt: 40,160,766,261.60 USD
Country: IDA only, Maximum Debt: 34,531,188,113.20 USD
Country: India, Maximum Debt: 31,923,507,000.80 USD
Country: Indonesia, Maximum Debt: 30,916,112,653.80 USD
Country: Kazakhstan, Maximum Debt: 27,482,093,686.40 USD
